In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [17]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [18]:
subsequences = 16
timesteps = X_train.shape[1]//subsequences

In [19]:
timesteps 

23

In [20]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [21]:
X_train.shape

(3499, 16, 1, 23, 1)

In [22]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=20)

In [23]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='mse')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=100, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/100
3499/3499 [==============================] - 11s 3ms/step - loss: 10354.8904 - val_loss: 9917.6729
Epoch 2/100
3499/3499 [==============================] - 10s 3ms/step - loss: 10284.0444 - val_loss: 9731.2516
Epoch 3/100
3499/3499 [==============================] - 10s 3ms/step - loss: 9104.4612 - val_loss: 2111.9728
Epoch 4/100
3499/3499 [==============================] - 10s 3ms/step - loss: 1380.3786 - val_loss: 471.6983
Epoch 5/100
3499/3499 [==============================] - 10s 3ms/step - loss: 505.0502 - val_loss: 438.6087
Epoch 6/100
3499/3499 [==============================] - 10s 3ms/step - loss: 429.2705 - val_loss: 337.3210
Epoch 7/100
3499/3499 [==============================] - 10s 3ms/step - loss: 310.1586 - val_loss: 189.1344
Epoch 8/100
3499/3499 [==============================] - 10s 3ms/step - loss: 259.2736 - val_loss: 195.7712
Epoch 9/100
3499/3499 [==============================] - 10s 3ms/step - loss: 24

3499/3499 [==============================] - 10s 3ms/step - loss: 109.1654 - val_loss: 84.0498
Epoch 77/100
3499/3499 [==============================] - 11s 3ms/step - loss: 106.2694 - val_loss: 76.0524
Epoch 78/100
3499/3499 [==============================] - 11s 3ms/step - loss: 108.3508 - val_loss: 75.4120
Epoch 79/100
3499/3499 [==============================] - 10s 3ms/step - loss: 108.1486 - val_loss: 77.4752
Epoch 80/100
3499/3499 [==============================] - 11s 3ms/step - loss: 107.2500 - val_loss: 73.8398
Epoch 81/100
3499/3499 [==============================] - 11s 3ms/step - loss: 105.6558 - val_loss: 83.5889
Epoch 82/100
3499/3499 [==============================] - 13s 4ms/step - loss: 105.0289 - val_loss: 77.5571
Epoch 83/100
3499/3499 [==============================] - 14s 4ms/step - loss: 101.8810 - val_loss: 74.5423
Epoch 84/100
3499/3499 [==============================] - 13s 4ms/step - loss: 101.7865 - val_loss: 72.2421
Epoch 85/100
3499/3499 [=================

In [24]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.5216817210295391,
 'Max Error': 41.92673828125001,
 'Mean Absolute Error': 6.012202614390917,
 'Mean Squared Error': 63.245239888853305,
 'Root Mean Squared Error': 7.952687589038897,
 'Median Absolute Error': 4.821315765380859,
 'R² Score': 0.5198275745452634,
 'Mean Absolute Percentage Error': 6.02487422763649}

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
